In [12]:
%reset -f
import pandas as pd
import requests
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [13]:
# Set gameweek
current_gw = 1

In [14]:
# API endpoints
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
fixtures = 'https://fantasy.premierleague.com/api/fixtures/'

In [15]:
response = requests.get(url)
fixtures = requests.get(fixtures).json()

if response.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {response.status_code}.')

The API request was successful.


In [16]:
teams = response.json()['teams']
team_mapping = {}

for team in teams:
    team_mapping[team['id']] = team['name']

In [17]:
team_difficulty = {
    "ARS": 7,
    "AVL": 5, 
    "BOU": 3,
    "BRE": 3,
    "BRI": 4,
    "BUR": 2,
    "CHE": 6,
    "CRY": 4,
    "EVE": 3,
    "FUL": 3, 
    "LIV": 7,
    "LEE": 2,
    "MCI": 6,
    "MUN": 4,
    "NEW": 4,
    "NFO": 4,
    "SUN": 2,
    "TOT": 4,
    "WHU": 3, 
    "WOL": 3
}

In [18]:
name_mapping = {
    "Arsenal": "ARS",
    "Aston Villa": "AVL", 
    "Bournemouth": "BOU",
    "Brentford": "BRE",
    "Brighton": "BRI",
    'Burnley': "BUR",
    "Chelsea": "CHE",
    "Crystal Palace": "CRY",
    "Everton": "EVE",
    "Fulham": "FUL", 
    "Leeds": "LEE",
    "Liverpool": "LIV",
    "Man City": "MCI",
    "Man Utd": "MUN",
    "Newcastle": "NEW",
    "Nott'm Forest": "NFO",
    "Spurs": "TOT",
    "Sunderland": "SUN",
    "West Ham": "WHU", 
    "Wolves": "WOL"
}

In [19]:
freq = defaultdict(int)
for v in team_difficulty.values():
    freq[v] += 1
    
df = pd.DataFrame.from_dict(freq, orient='index', columns=['Count'])
df.index.name = 'Difficulty'
df = df.sort_index(ascending=False)

df

,Count
Difficulty,
7,2
6,2
5,1
4,6
3,6
2,3


In [20]:
data = []

for match in fixtures:
    if 'event' not in match or match['event'] is None:
        continue
    elif match['event'] >= current_gw and match['event'] <= (current_gw + 10):
        home_team = team_mapping[match['team_h']]
        away_team = team_mapping[match['team_a']]
    
        data.append({
          'event': match['event'],
          'Team': home_team,
          'Team2': away_team
        })

        data.append({
          'event': match['event'],
          'Team': away_team,
          'Team2': home_team
        })

df = pd.DataFrame(data)

# Shorten team names
df["Team"] = df["Team"].map(lambda x: name_mapping[x])
df["Team2"] = df["Team2"].map(lambda x: name_mapping[x])

df['Team2'] = df.groupby(['event', 'Team'])['Team2'].transform(lambda x: ', '.join(x))
df.drop_duplicates(subset=['event', 'Team'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [21]:
fdr = df.pivot(index='Team', columns='event', values='Team2')
fdr.columns = [f'GW{col}' for col in fdr.columns]
fdr.fillna('-', inplace = True)

fdr

,GW1,GW2,GW3,GW4,GW5,GW6,GW7,GW8,GW9,GW10,GW11
Team,,,,,,,,,,,
ARS,MUN,LEE,LIV,NFO,MCI,NEW,WHU,FUL,CRY,BUR,SUN
AVL,NEW,BRE,CRY,EVE,SUN,FUL,BUR,TOT,MCI,LIV,BOU
BOU,LIV,WOL,TOT,BRI,NEW,LEE,FUL,CRY,NFO,MCI,AVL
BRE,NFO,AVL,SUN,CHE,FUL,MUN,MCI,WHU,LIV,CRY,NEW
BRI,FUL,EVE,MCI,BOU,TOT,CHE,WOL,NEW,MUN,LEE,CRY
BUR,TOT,SUN,MUN,LIV,NFO,MCI,AVL,LEE,WOL,ARS,WHU
CHE,CRY,WHU,FUL,BRE,MUN,BRI,LIV,NFO,SUN,TOT,WOL
CRY,CHE,NFO,AVL,SUN,WHU,LIV,EVE,BOU,ARS,BRE,BRI
EVE,LEE,BRI,WOL,AVL,LIV,WHU,CRY,MCI,TOT,SUN,FUL


In [22]:
def gradient_color(val):
    if val in team_difficulty:
        difficulty = team_difficulty[val]
        norm = plt.Normalize(min(team_difficulty.values()), max(team_difficulty.values()))
        cmap = plt.cm.RdYlGn_r 
        color = mcolors.to_hex(cmap(norm(difficulty)))
        return f"background-color: {color}"
    return ""

styled_fdr = fdr.style.applymap(gradient_color)
styled_fdr

/var/folders/_k/hgt2zf0s18x5lvv1gwk899pc0000gn/T/ipykernel_52515/2233052785.py:10: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_fdr = fdr.style.applymap(gradient_color)


,GW1,GW2,GW3,GW4,GW5,GW6,GW7,GW8,GW9,GW10,GW11
Team,,,,,,,,,,,
ARS,MUN,LEE,LIV,NFO,MCI,NEW,WHU,FUL,CRY,BUR,SUN
AVL,NEW,BRE,CRY,EVE,SUN,FUL,BUR,TOT,MCI,LIV,BOU
BOU,LIV,WOL,TOT,BRI,NEW,LEE,FUL,CRY,NFO,MCI,AVL
BRE,NFO,AVL,SUN,CHE,FUL,MUN,MCI,WHU,LIV,CRY,NEW
BRI,FUL,EVE,MCI,BOU,TOT,CHE,WOL,NEW,MUN,LEE,CRY
BUR,TOT,SUN,MUN,LIV,NFO,MCI,AVL,LEE,WOL,ARS,WHU
CHE,CRY,WHU,FUL,BRE,MUN,BRI,LIV,NFO,SUN,TOT,WOL
CRY,CHE,NFO,AVL,SUN,WHU,LIV,EVE,BOU,ARS,BRE,BRI
EVE,LEE,BRI,WOL,AVL,LIV,WHU,CRY,MCI,TOT,SUN,FUL
